                                    Welcome to Experimenting with Various Models


   I trained various models on the data through this process. The purpose of this was to find the best model architecture for the goal of this project, which is object detection and classification on our dataset of playing cards. To facilitate a speedier exploration I scaled down the dataset that would be used. The reason for this is that training previous Neural Networks on this data gave us an estimated 30 days to run on this laptop; the process was aborted and ran on a local gpu which still took 4 days. The dataset was shrunk from 14,000 images to 500. 
   
   
   Architectures to Explore:
   
   1. Yolov5
   
   
  2. Faster R-CNN

   After training both of these models the model that was the most accurate was Yolov5. It was far and away the best model between the two. 
    This will be the architecture that will be used in further projects. 

The code for these models are below.

























In [19]:
import numpy as np
import pandas as pd
import torch

import os

import cv2

from matplotlib import pyplot as plt

%matplotlib inline

The full dataset was reduced from 14,000 to 500 to facilitate faster training.

The first method that I will use to analyze this data will be yolov5. This stands for "You Only Look Once" version 5. 
This is a popular architecture for lightning fast image detection and image classification. 

YOLO V 5

In [9]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

Using cache found in C:\Users\Obama/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-10-6 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
!python train.py --img 416 --batch 16 --epochs 400 --data dataset.yaml --weights yolov5s.pt

The next approach will be the Faster R-CNN approach. A R-CNN is a Region based Convolutional Neural Network.

In [215]:
import os
import torch
from PIL import Image


In [77]:

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.annotations = list(sorted(os.listdir(os.path.join(root, "labels"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        ann_path = os.path.join(self.root, "labels", self.annotations[idx])
        img = Image.open(img_path).convert("RGB")
        
        # Read the annotation file
        with open(ann_path) as f:
            annotations = f.readlines()
        
        boxes = []
        labels = []
        img_width, img_height = img.size

        for annotation in annotations:
            parts = annotation.strip().split()
            label = int(parts[0])
            center_x = float(parts[1]) * img_width
            center_y = float(parts[2]) * img_height
            width = float(parts[3]) * img_width
            height = float(parts[4]) * img_height

            xmin = center_x - width / 2
            ymin = center_y - height / 2
            xmax = center_x + width / 2
            ymax = center_y + height / 2

            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        if self.transforms:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [78]:
from torch.utils.data import DataLoader

In [79]:
import torchvision.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [80]:
# Use our dataset and defined transformations
dataset = CustomDataset(root='', transforms=get_transform(train=True))

In [25]:
len(dataset.imgs)

500

In [81]:
dataset.imgs[0]

'000090528_jpg.rf.d50e89610e5c97c61632c290692f3e75.jpg'

In [82]:
from torchvision import transforms

In [83]:
listr = dataset.imgs

In [29]:
len(listr)

500

In [84]:

transformr = transforms.ToTensor()

In [85]:
listr

['000090528_jpg.rf.d50e89610e5c97c61632c290692f3e75.jpg',
 '000233645_jpg.rf.843259f9e28b9e485f219f8bfa2c5cdf.jpg',
 '000452673_jpg.rf.691b39608109cde124380edb13c2a485.jpg',
 '000554394_jpg.rf.8be18f32226fb0d2208dddd4349077ba.jpg',
 '000609988_jpg.rf.53fdbac0baabace7c9ca457daac7e960.jpg',
 '000618838_jpg.rf.e1a6260c04b2e44c95c94280963812d6.jpg',
 '000807742_jpg.rf.6730af8a9604c80ade38ded7a5f7cec3.jpg',
 '000868268_jpg.rf.154d4050602f82b6017f95afaa8022e5.jpg',
 '000898488_jpg.rf.7f8eceb05724a0fdbc591ac1872126e3.jpg',
 '000933090_jpg.rf.4b7900a612be70d47dae80ffab16a667.jpg',
 '000958849_jpg.rf.080b6270c4d4ff260774c0bf5c780f44.jpg',
 '001012368_jpg.rf.f73c26509b49e556cac5f42087d2839c.jpg',
 '001024107_jpg.rf.f0ffe79a74d7514bcecd86a119a01f38.jpg',
 '001167476_jpg.rf.76546eae647d75a8b1e9f28a314155e7.jpg',
 '001220385_jpg.rf.4985459ef04c6a9202391af01fd3d117.jpg',
 '001324849_jpg.rf.b2dc884dfe19cce223a428c3be24d7b8.jpg',
 '001477182_jpg.rf.1df1cfaef97f6e8552f2c72e3d31d5f3.jpg',
 '001874057_jp

In [86]:

f_data = [item for idx, item in enumerate(listr) if idx not in con_list]


In [87]:
len(f_data)

405

In [88]:
tensor_list = []


for image_path in f_data:
    image = Image.open(os.path.join('data\\images\\', image_path)).convert("RGB")
    image_tensor = transformr(image)
    tensor_list.append(image_tensor)

# Stack all image tensors into a single tensor with dimensions [batch_size, channels, height, width]
result_tensor = torch.stack(tensor_list)


In [89]:
result_tensor.shape

torch.Size([405, 3, 416, 416])

In [16]:
# Now get labels in the form of tensors

In [33]:
labels_list = dataset.annotations

In [34]:
len(labels_list)

500

In [35]:
tensor_listol = []

In [36]:
for file_path in labels_list:
    
    with open(os.path.join("data\\labels\\", file_path), 'r') as file:
        lines = file.readlines()
        
        relee = []
        
        for line in lines:
        
            numbers_str = line.strip().split()
        
        
            float_numbers = [float(num_str) for num_str in numbers_str]
            txt_tensor = torch.tensor(float_numbers, dtype=torch.float32)
            relee.append(txt_tensor)
            
        tensor_listol.append(relee)
        
        
        
        
        
#result_txt_tensor = torch.stack(tensor_listol)

In [38]:
tensor_listol

[[tensor([5.1000e+01, 4.6034e-01, 6.9351e-01, 6.4904e-02, 4.2067e-02]),
  tensor([37.0000,  0.6442,  0.5012,  0.0637,  0.0889]),
  tensor([37.0000,  0.6142,  0.0745,  0.0649,  0.0889])],
 [tensor([17.0000,  0.4808,  0.7933,  0.0637,  0.0553]),
  tensor([24.0000,  0.4459,  0.7380,  0.0661,  0.0529]),
  tensor([8.0000, 0.4303, 0.6755, 0.0685, 0.0433]),
  tensor([8.0000, 0.5974, 0.4507, 0.0697, 0.0433])],
 [tensor([3.6000e+01, 4.5913e-01, 3.9062e-01, 6.6106e-02, 3.4856e-02]),
  tensor([23.0000,  0.4663,  0.4399,  0.0685,  0.0325]),
  tensor([4.4000e+01, 4.5793e-01, 5.0841e-01, 6.8510e-02, 3.6058e-02]),
  tensor([4.4000e+01, 2.1514e-01, 6.4663e-01, 6.9712e-02, 3.4856e-02])],
 [tensor([30.0000,  0.6611,  0.1671,  0.0589,  0.0517]),
  tensor([31.0000,  0.6851,  0.2115,  0.0637,  0.0505]),
  tensor([6.0000, 0.7079, 0.2957, 0.0649, 0.0361]),
  tensor([6.0000, 0.5276, 0.5108, 0.0685, 0.0373])],
 [tensor([28.0000,  0.6010,  0.7656,  0.0469,  0.0649]),
  tensor([5.0000, 0.5565, 0.7344, 0.0517, 0.

In [40]:
tensor_listol[0][0].shape

torch.Size([5])

In [42]:
len(tensor_listol[0])

3

In [60]:
outer_d = 5
for i in tensor_listol:
    c = len(i)
    if c <= outer_d:
        outer_d = c

In [61]:
outer_d

2

In [55]:
inner_d = 5
for i in tensor_listol:
    for x in i:
        l = x.shape
        for dim in l:
            if dim <= inner_d:
                inner_d = dim

In [57]:
inner_d

5

In [168]:
tensor_listol[23][0].shape

torch.Size([5])

In [149]:
    with open(os.path.join("data\\labels\\", file_path), 'r') as file:
        lines = file.readlines()
        
        relee = []
        
        for line in lines:
        
            numbers_str = line.strip().split()
            print(numbers_str)
        

['26', '0.4795673076923077', '0.41586538461538464', '0.06610576923076923', '0.040865384615384616']
['49', '0.47836538461538464', '0.34975961538461536', '0.06850961538461539', '0.030048076923076924']
['25', '0.4879807692307692', '0.28725961538461536', '0.07091346153846154', '0.037259615384615384']
['25', '0.7367788461538461', '0.15745192307692307', '0.0733173076923077', '0.037259615384615384']


In [64]:
counter = 0 
list_2 =[]

for i in tensor_listol:
    c = len(i)
    if c == 2:
        list_2.append(counter)
    counter += 1

In [65]:
list_2

[83, 139, 163, 352, 443]

In [66]:
counter = 0 
list_3 =[]

for i in tensor_listol:
    c = len(i)
    if c == 3:
        list_3.append(counter)
    counter += 1

In [67]:
list_3

[0,
 10,
 13,
 19,
 28,
 30,
 32,
 36,
 38,
 59,
 62,
 64,
 66,
 69,
 70,
 74,
 85,
 98,
 100,
 101,
 120,
 123,
 126,
 135,
 136,
 142,
 146,
 161,
 162,
 167,
 180,
 188,
 190,
 202,
 205,
 218,
 220,
 221,
 222,
 226,
 229,
 236,
 239,
 247,
 250,
 257,
 259,
 262,
 266,
 267,
 276,
 294,
 296,
 299,
 302,
 306,
 317,
 322,
 323,
 324,
 335,
 338,
 343,
 348,
 349,
 355,
 363,
 368,
 387,
 390,
 391,
 398,
 399,
 405,
 406,
 408,
 409,
 414,
 415,
 423,
 432,
 435,
 437,
 439,
 447,
 469,
 484,
 485,
 486,
 495]

In [69]:
con_list = list_2+list_3

In [70]:
filtered_data = [item for idx, item in enumerate(tensor_listol) if idx not in con_list]


In [71]:
filtered_data

[[tensor([17.0000,  0.4808,  0.7933,  0.0637,  0.0553]),
  tensor([24.0000,  0.4459,  0.7380,  0.0661,  0.0529]),
  tensor([8.0000, 0.4303, 0.6755, 0.0685, 0.0433]),
  tensor([8.0000, 0.5974, 0.4507, 0.0697, 0.0433])],
 [tensor([3.6000e+01, 4.5913e-01, 3.9062e-01, 6.6106e-02, 3.4856e-02]),
  tensor([23.0000,  0.4663,  0.4399,  0.0685,  0.0325]),
  tensor([4.4000e+01, 4.5793e-01, 5.0841e-01, 6.8510e-02, 3.6058e-02]),
  tensor([4.4000e+01, 2.1514e-01, 6.4663e-01, 6.9712e-02, 3.4856e-02])],
 [tensor([30.0000,  0.6611,  0.1671,  0.0589,  0.0517]),
  tensor([31.0000,  0.6851,  0.2115,  0.0637,  0.0505]),
  tensor([6.0000, 0.7079, 0.2957, 0.0649, 0.0361]),
  tensor([6.0000, 0.5276, 0.5108, 0.0685, 0.0373])],
 [tensor([28.0000,  0.6010,  0.7656,  0.0469,  0.0649]),
  tensor([5.0000, 0.5565, 0.7344, 0.0517, 0.0601]),
  tensor([6.0000, 0.4964, 0.6683, 0.0589, 0.0505]),
  tensor([6.0000, 0.5685, 0.3954, 0.0601, 0.0529])],
 [tensor([11.0000,  0.6370,  0.3606,  0.0865,  0.0901]),
  tensor([49.0000

In [74]:
inner_stacked = [torch.stack(inner_list) for inner_list in filtered_data]

# Then stack the resulting tensors along another new dimension
final_tensor = torch.stack(inner_stacked)

In [75]:
final_tensor

tensor([[[1.7000e+01, 4.8077e-01, 7.9327e-01, 6.3702e-02, 5.5288e-02],
         [2.4000e+01, 4.4591e-01, 7.3798e-01, 6.6106e-02, 5.2885e-02],
         [8.0000e+00, 4.3029e-01, 6.7548e-01, 6.8510e-02, 4.3269e-02],
         [8.0000e+00, 5.9736e-01, 4.5072e-01, 6.9712e-02, 4.3269e-02]],

        [[3.6000e+01, 4.5913e-01, 3.9062e-01, 6.6106e-02, 3.4856e-02],
         [2.3000e+01, 4.6635e-01, 4.3990e-01, 6.8510e-02, 3.2452e-02],
         [4.4000e+01, 4.5793e-01, 5.0841e-01, 6.8510e-02, 3.6058e-02],
         [4.4000e+01, 2.1514e-01, 6.4663e-01, 6.9712e-02, 3.4856e-02]],

        [[3.0000e+01, 6.6106e-01, 1.6707e-01, 5.8894e-02, 5.1683e-02],
         [3.1000e+01, 6.8510e-01, 2.1154e-01, 6.3702e-02, 5.0481e-02],
         [6.0000e+00, 7.0793e-01, 2.9567e-01, 6.4904e-02, 3.6058e-02],
         [6.0000e+00, 5.2764e-01, 5.1082e-01, 6.8510e-02, 3.7260e-02]],

        ...,

        [[3.1000e+01, 7.0433e-01, 7.3798e-01, 7.4519e-02, 9.1346e-02],
         [1.8000e+01, 6.3942e-01, 6.8269e-01, 7.9327e-02,

In [90]:
final_tensor.shape

torch.Size([405, 4, 5])

In [ ]:
#images
result_tensor
#labels
final_tensor

In [ ]:
# Now create the model

In [91]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset

In [100]:
boc = []

for i in final_tensor:
    temp = []
    for x in i:
        temp.append(x[0])
    boc.append(temp)
        
    

In [96]:
final_tensor[0]

tensor([[17.0000,  0.4808,  0.7933,  0.0637,  0.0553],
        [24.0000,  0.4459,  0.7380,  0.0661,  0.0529],
        [ 8.0000,  0.4303,  0.6755,  0.0685,  0.0433],
        [ 8.0000,  0.5974,  0.4507,  0.0697,  0.0433]])

In [98]:
final_tensor[0][0]

tensor([17.0000,  0.4808,  0.7933,  0.0637,  0.0553])

In [97]:
final_tensor[0][2][0]

tensor(8.)

In [101]:
boc

[[tensor(17.), tensor(24.), tensor(8.), tensor(8.)],
 [tensor(36.), tensor(23.), tensor(44.), tensor(44.)],
 [tensor(30.), tensor(31.), tensor(6.), tensor(6.)],
 [tensor(28.), tensor(5.), tensor(6.), tensor(6.)],
 [tensor(11.), tensor(49.), tensor(21.), tensor(21.)],
 [tensor(38.), tensor(38.), tensor(51.), tensor(51.)],
 [tensor(7.), tensor(46.), tensor(40.), tensor(40.)],
 [tensor(43.), tensor(2.), tensor(32.), tensor(32.)],
 [tensor(14.), tensor(39.), tensor(51.), tensor(51.)],
 [tensor(5.), tensor(5.), tensor(40.), tensor(40.)],
 [tensor(50.), tensor(48.), tensor(33.), tensor(33.)],
 [tensor(8.), tensor(9.), tensor(0.), tensor(0.)],
 [tensor(46.), tensor(3.), tensor(23.), tensor(23.)],
 [tensor(41.), tensor(14.), tensor(38.), tensor(38.)],
 [tensor(39.), tensor(37.), tensor(25.), tensor(25.)],
 [tensor(14.), tensor(14.), tensor(46.), tensor(46.)],
 [tensor(40.), tensor(19.), tensor(17.), tensor(17.)],
 [tensor(37.), tensor(37.), tensor(29.), tensor(29.)],
 [tensor(34.), tensor(15.)

In [102]:
inner_ked = [torch.stack(inner_list) for inner_list in boc]

# Then stack the resulting tensors along another new dimension
box = torch.stack(inner_ked)

In [103]:
#box

tensor([[17., 24.,  8.,  8.],
        [36., 23., 44., 44.],
        [30., 31.,  6.,  6.],
        ...,
        [31., 18., 29., 29.],
        [35., 35.,  4.,  4.],
        [26., 49., 25., 25.]])

In [ ]:
#images
result_tensor
#final class labels in tensor
box
#final box labels in tensor
t

In [297]:
ids = box.tolist()

In [300]:
ids

[[17.0, 24.0, 8.0, 8.0],
 [36.0, 23.0, 44.0, 44.0],
 [30.0, 31.0, 6.0, 6.0],
 [28.0, 5.0, 6.0, 6.0],
 [11.0, 49.0, 21.0, 21.0],
 [38.0, 38.0, 51.0, 51.0],
 [7.0, 46.0, 40.0, 40.0],
 [43.0, 2.0, 32.0, 32.0],
 [14.0, 39.0, 51.0, 51.0],
 [5.0, 5.0, 40.0, 40.0],
 [50.0, 48.0, 33.0, 33.0],
 [8.0, 9.0, 0.0, 0.0],
 [46.0, 3.0, 23.0, 23.0],
 [41.0, 14.0, 38.0, 38.0],
 [39.0, 37.0, 25.0, 25.0],
 [14.0, 14.0, 46.0, 46.0],
 [40.0, 19.0, 17.0, 17.0],
 [37.0, 37.0, 29.0, 29.0],
 [34.0, 15.0, 16.0, 16.0],
 [4.0, 7.0, 26.0, 26.0],
 [49.0, 19.0, 12.0, 12.0],
 [28.0, 28.0, 47.0, 47.0],
 [47.0, 47.0, 3.0, 3.0],
 [49.0, 49.0, 29.0, 29.0],
 [19.0, 36.0, 35.0, 35.0],
 [51.0, 24.0, 10.0, 10.0],
 [48.0, 43.0, 36.0, 36.0],
 [7.0, 7.0, 39.0, 39.0],
 [32.0, 32.0, 43.0, 43.0],
 [21.0, 40.0, 41.0, 41.0],
 [11.0, 47.0, 11.0, 11.0],
 [31.0, 22.0, 30.0, 30.0],
 [12.0, 12.0, 27.0, 27.0],
 [17.0, 17.0, 25.0, 25.0],
 [38.0, 38.0, 20.0, 20.0],
 [40.0, 25.0, 39.0, 39.0],
 [33.0, 43.0, 44.0, 44.0],
 [7.0, 16.0, 49.0, 49.0

In [301]:
dsf = []
for i in ids:
    wejkf = []
    for x in i:
        wejkf.append(int(x))
    dsf.append(wejkf)

In [302]:
dsf

[[17, 24, 8, 8],
 [36, 23, 44, 44],
 [30, 31, 6, 6],
 [28, 5, 6, 6],
 [11, 49, 21, 21],
 [38, 38, 51, 51],
 [7, 46, 40, 40],
 [43, 2, 32, 32],
 [14, 39, 51, 51],
 [5, 5, 40, 40],
 [50, 48, 33, 33],
 [8, 9, 0, 0],
 [46, 3, 23, 23],
 [41, 14, 38, 38],
 [39, 37, 25, 25],
 [14, 14, 46, 46],
 [40, 19, 17, 17],
 [37, 37, 29, 29],
 [34, 15, 16, 16],
 [4, 7, 26, 26],
 [49, 19, 12, 12],
 [28, 28, 47, 47],
 [47, 47, 3, 3],
 [49, 49, 29, 29],
 [19, 36, 35, 35],
 [51, 24, 10, 10],
 [48, 43, 36, 36],
 [7, 7, 39, 39],
 [32, 32, 43, 43],
 [21, 40, 41, 41],
 [11, 47, 11, 11],
 [31, 22, 30, 30],
 [12, 12, 27, 27],
 [17, 17, 25, 25],
 [38, 38, 20, 20],
 [40, 25, 39, 39],
 [33, 43, 44, 44],
 [7, 16, 49, 49],
 [46, 2, 41, 41],
 [30, 34, 40, 40],
 [18, 27, 0, 0],
 [17, 44, 7, 7],
 [18, 18, 41, 41],
 [44, 34, 16, 16],
 [36, 42, 26, 26],
 [38, 5, 25, 25],
 [15, 48, 10, 10],
 [10, 10, 25, 25],
 [40, 17, 40, 40],
 [11, 11, 0, 0],
 [11, 18, 48, 48],
 [13, 24, 34, 34],
 [38, 38, 24, 24],
 [45, 45, 16, 16],
 [41,

In [307]:
bief = []
for i in dsf:
    bief.append(torch.Tensor(i))

In [308]:
bief

[tensor([17., 24.,  8.,  8.]),
 tensor([36., 23., 44., 44.]),
 tensor([30., 31.,  6.,  6.]),
 tensor([28.,  5.,  6.,  6.]),
 tensor([11., 49., 21., 21.]),
 tensor([38., 38., 51., 51.]),
 tensor([ 7., 46., 40., 40.]),
 tensor([43.,  2., 32., 32.]),
 tensor([14., 39., 51., 51.]),
 tensor([ 5.,  5., 40., 40.]),
 tensor([50., 48., 33., 33.]),
 tensor([8., 9., 0., 0.]),
 tensor([46.,  3., 23., 23.]),
 tensor([41., 14., 38., 38.]),
 tensor([39., 37., 25., 25.]),
 tensor([14., 14., 46., 46.]),
 tensor([40., 19., 17., 17.]),
 tensor([37., 37., 29., 29.]),
 tensor([34., 15., 16., 16.]),
 tensor([ 4.,  7., 26., 26.]),
 tensor([49., 19., 12., 12.]),
 tensor([28., 28., 47., 47.]),
 tensor([47., 47.,  3.,  3.]),
 tensor([49., 49., 29., 29.]),
 tensor([19., 36., 35., 35.]),
 tensor([51., 24., 10., 10.]),
 tensor([48., 43., 36., 36.]),
 tensor([ 7.,  7., 39., 39.]),
 tensor([32., 32., 43., 43.]),
 tensor([21., 40., 41., 41.]),
 tensor([11., 47., 11., 11.]),
 tensor([31., 22., 30., 30.]),
 tensor([12.

In [336]:
class pyuret(Dataset):
    def __init__(self, images, boxes, classifiers):
        self.images = images
        self.boxes = boxes
        self.classifiers = classifiers
        self.num_samples = 405

    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        image = self.images[idx]
        boxes = self.boxes[idx]
        classifiers = self.classifiers[idx]
        
              
        
        targets = []
        counter = 0
        for classifier_params in classifiers:
            target = {}
            target['boxes'] = uh_oh_stinky[counter]
            target['labels'] = classifier_params
            targets.append(target)
            counter += 1
        
        return image, targets

In [343]:
images

[tensor([[[0.6039, 0.5686, 0.5059,  ..., 0.3137, 0.3098, 0.3059],
          [0.5529, 0.5176, 0.4588,  ..., 0.3333, 0.3255, 0.3216],
          [0.4549, 0.4353, 0.3843,  ..., 0.3373, 0.3333, 0.3294],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5608, 0.7294, 0.7647],
          [0.0314, 0.0392, 0.0549,  ..., 0.3686, 0.5255, 0.5843],
          [0.0196, 0.0314, 0.0510,  ..., 0.2275, 0.3608, 0.4431]],
 
         [[0.5137, 0.4902, 0.4235,  ..., 0.3059, 0.3020, 0.2980],
          [0.4627, 0.4392, 0.3843,  ..., 0.3255, 0.3176, 0.3137],
          [0.3725, 0.3529, 0.3098,  ..., 0.3294, 0.3255, 0.3216],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5020, 0.6549, 0.6902],
          [0.0314, 0.0392, 0.0549,  ..., 0.3098, 0.4510, 0.5098],
          [0.0196, 0.0314, 0.0510,  ..., 0.1686, 0.2863, 0.3686]],
 
         [[0.4902, 0.4627, 0.4039,  ..., 0.3098, 0.3059, 0.3020],
          [0.4392, 0.4118, 0.3608,  ..., 0.3294, 0.3216, 0.3176],
          [0.3529, 0.3333, 0.2863,  ...,

In [344]:
shbi = pyuret(images, uh_oh_stinky, bief)

In [345]:
type(images)

list

In [340]:
bief

[tensor([17., 24.,  8.,  8.]),
 tensor([36., 23., 44., 44.]),
 tensor([30., 31.,  6.,  6.]),
 tensor([28.,  5.,  6.,  6.]),
 tensor([11., 49., 21., 21.]),
 tensor([38., 38., 51., 51.]),
 tensor([ 7., 46., 40., 40.]),
 tensor([43.,  2., 32., 32.]),
 tensor([14., 39., 51., 51.]),
 tensor([ 5.,  5., 40., 40.]),
 tensor([50., 48., 33., 33.]),
 tensor([8., 9., 0., 0.]),
 tensor([46.,  3., 23., 23.]),
 tensor([41., 14., 38., 38.]),
 tensor([39., 37., 25., 25.]),
 tensor([14., 14., 46., 46.]),
 tensor([40., 19., 17., 17.]),
 tensor([37., 37., 29., 29.]),
 tensor([34., 15., 16., 16.]),
 tensor([ 4.,  7., 26., 26.]),
 tensor([49., 19., 12., 12.]),
 tensor([28., 28., 47., 47.]),
 tensor([47., 47.,  3.,  3.]),
 tensor([49., 49., 29., 29.]),
 tensor([19., 36., 35., 35.]),
 tensor([51., 24., 10., 10.]),
 tensor([48., 43., 36., 36.]),
 tensor([ 7.,  7., 39., 39.]),
 tensor([32., 32., 43., 43.]),
 tensor([21., 40., 41., 41.]),
 tensor([11., 47., 11., 11.]),
 tensor([31., 22., 30., 30.]),
 tensor([12.

In [108]:
lab = []

for i in final_tensor:
    tep = []
    for x in i:
        oop = []
        oop.append(x[1])
        oop.append(x[2])
        oop.append(x[3])
        oop.append(x[4])
        tep.append(oop)
        
    
    lab.append(tep)
        
    

In [110]:
lab

[[[tensor(0.4808), tensor(0.7933), tensor(0.0637), tensor(0.0553)],
  [tensor(0.4459), tensor(0.7380), tensor(0.0661), tensor(0.0529)],
  [tensor(0.4303), tensor(0.6755), tensor(0.0685), tensor(0.0433)],
  [tensor(0.5974), tensor(0.4507), tensor(0.0697), tensor(0.0433)]],
 [[tensor(0.4591), tensor(0.3906), tensor(0.0661), tensor(0.0349)],
  [tensor(0.4663), tensor(0.4399), tensor(0.0685), tensor(0.0325)],
  [tensor(0.4579), tensor(0.5084), tensor(0.0685), tensor(0.0361)],
  [tensor(0.2151), tensor(0.6466), tensor(0.0697), tensor(0.0349)]],
 [[tensor(0.6611), tensor(0.1671), tensor(0.0589), tensor(0.0517)],
  [tensor(0.6851), tensor(0.2115), tensor(0.0637), tensor(0.0505)],
  [tensor(0.7079), tensor(0.2957), tensor(0.0649), tensor(0.0361)],
  [tensor(0.5276), tensor(0.5108), tensor(0.0685), tensor(0.0373)]],
 [[tensor(0.6010), tensor(0.7656), tensor(0.0469), tensor(0.0649)],
  [tensor(0.5565), tensor(0.7344), tensor(0.0517), tensor(0.0601)],
  [tensor(0.4964), tensor(0.6683), tensor(0.0

In [325]:
class pyDataset(Dataset):
    def __init__(self, images, boxes, classifiers):
        self.images = images
        self.boxes = boxes
        self.classifiers = classifiers
        self.num_samples = images.size(0)
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        image = self.images[idx]
        boxes = self.boxes[idx]
        classifiers = self.classifiers[idx]
        
              
        
        targets = []
        counter = 0
        for classifier_params in classifiers:
            target = {}
            target['boxes'] = uh_oh_stinky[counter]
            target['labels'] = classifier_params
            targets.append(target)
            counter += 1
        
        return image, targets

In [258]:
dataclass = pyDataset(result_tensor, uh_oh_stinky, box)

In [262]:
dataclass[0]

(tensor([[[0.6039, 0.5686, 0.5059,  ..., 0.3137, 0.3098, 0.3059],
          [0.5529, 0.5176, 0.4588,  ..., 0.3333, 0.3255, 0.3216],
          [0.4549, 0.4353, 0.3843,  ..., 0.3373, 0.3333, 0.3294],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5608, 0.7294, 0.7647],
          [0.0314, 0.0392, 0.0549,  ..., 0.3686, 0.5255, 0.5843],
          [0.0196, 0.0314, 0.0510,  ..., 0.2275, 0.3608, 0.4431]],
 
         [[0.5137, 0.4902, 0.4235,  ..., 0.3059, 0.3020, 0.2980],
          [0.4627, 0.4392, 0.3843,  ..., 0.3255, 0.3176, 0.3137],
          [0.3725, 0.3529, 0.3098,  ..., 0.3294, 0.3255, 0.3216],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5020, 0.6549, 0.6902],
          [0.0314, 0.0392, 0.0549,  ..., 0.3098, 0.4510, 0.5098],
          [0.0196, 0.0314, 0.0510,  ..., 0.1686, 0.2863, 0.3686]],
 
         [[0.4902, 0.4627, 0.4039,  ..., 0.3098, 0.3059, 0.3020],
          [0.4392, 0.4118, 0.3608,  ..., 0.3294, 0.3216, 0.3176],
          [0.3529, 0.3333, 0.2863,  ...,

In [272]:
from torchvision import transforms
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(dataclass, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [273]:
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn

model = fasterrcnn_resnet50_fpn(pretrained=True)

In [274]:
# Replace the classifier with a new one, which has num_classes which is user-defined
num_classes = 52  # Example: if your dataset has 3 classes (background + 2 objects)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

In [204]:
yyy = t.tolist()

In [206]:
t.shape

torch.Size([405, 4, 4])

In [205]:
yyy

[[[0.48076921701431274,
   0.7932692170143127,
   0.06370192021131516,
   0.05528846010565758],
  [0.4459134638309479,
   0.7379807829856873,
   0.06610576808452606,
   0.05288461595773697],
  [0.4302884638309479,
   0.6754807829856873,
   0.06850961595773697,
   0.04326923191547394],
  [0.5973557829856873,
   0.4507211446762085,
   0.06971153616905212,
   0.04326923191547394]],
 [[0.45913460850715637, 0.390625, 0.06610576808452606, 0.03485576808452606],
  [0.4663461446762085,
   0.4399038553237915,
   0.06850961595773697,
   0.032451923936605453],
  [0.4579326808452606,
   0.5084134340286255,
   0.06850961595773697,
   0.036057692021131516],
  [0.21514423191547394,
   0.6466346383094788,
   0.06971153616905212,
   0.03485576808452606]],
 [[0.661057710647583,
   0.16706730425357819,
   0.05889423191547394,
   0.051682692021131516],
  [0.6850961446762085,
   0.21153846383094788,
   0.06370192021131516,
   0.05048076808452606],
  [0.707932710647583,
   0.29567307233810425,
   0.064903847

In [238]:
def computations(prophet):
    xcen = prophet[0]*416
    ycen = prophet[1]*416
    width = prophet[2]*416
    height = prophet[3]*416
    
    x_min = (xcen - (width/2))
    x_max = (xcen + (width/2))
    y_min = (ycen - (height/2))
    y_max = (ycen + (height/2))
    
    of_the_king = [x_min, y_min, x_max, y_max]
    return of_the_king

    

uh = []
for i in yyy:
    oh = []
    for x in i:
        stinky = computations(x)
        oh.append(stinky)
        
    uh.append(oh)
        
        
        

In [212]:
uh

[[[186.74999487400055,
   318.4999945759773,
   213.24999368190765,
   341.4999939799309],
  [171.7500011920929,
   296.0000056028366,
   199.25000071525574,
   318.0000058412552],
  [164.75000083446503,
   272.0000054836273,
   193.2500010728836,
   290.0000059604645],
  [234.00000619888306,
   178.49999594688416,
   263.00000524520874,
   196.4999964237213]],
 [[177.24999737739563,
   155.25000023841858,
   204.74999690055847,
   169.74999976158142],
  [179.74999606609344,
   176.25000363588333,
   208.24999630451202,
   189.7500039935112],
  [176.24999511241913,
   203.99998861551285,
   204.7499953508377,
   218.99998849630356],
  [75.00000095367432, 261.75000977516174, 104.0, 276.2500092983246]],
 [[262.75000739097595,
   58.74999862909317,
   287.2500078678131,
   80.24999850988388],
  [271.7499967813492, 77.5000011920929, 298.2499955892563, 98.50000071525574],
  [281.00000727176666,
   115.49999815225601,
   308.0000079870224,
   130.49999803304672],
  [205.2499941587448,
   204

In [217]:
uh_oh_stinky = torch.Tensor(uh)

In [ ]:
# images
result_tensor

#final class labels
box

# the bounding boxes in the correct format
uh_oh_stinky

In [393]:
images = list(image for image in dataclass.images)

In [395]:
images

[tensor([[[0.6039, 0.5686, 0.5059,  ..., 0.3137, 0.3098, 0.3059],
          [0.5529, 0.5176, 0.4588,  ..., 0.3333, 0.3255, 0.3216],
          [0.4549, 0.4353, 0.3843,  ..., 0.3373, 0.3333, 0.3294],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5608, 0.7294, 0.7647],
          [0.0314, 0.0392, 0.0549,  ..., 0.3686, 0.5255, 0.5843],
          [0.0196, 0.0314, 0.0510,  ..., 0.2275, 0.3608, 0.4431]],
 
         [[0.5137, 0.4902, 0.4235,  ..., 0.3059, 0.3020, 0.2980],
          [0.4627, 0.4392, 0.3843,  ..., 0.3255, 0.3176, 0.3137],
          [0.3725, 0.3529, 0.3098,  ..., 0.3294, 0.3255, 0.3216],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5020, 0.6549, 0.6902],
          [0.0314, 0.0392, 0.0549,  ..., 0.3098, 0.4510, 0.5098],
          [0.0196, 0.0314, 0.0510,  ..., 0.1686, 0.2863, 0.3686]],
 
         [[0.4902, 0.4627, 0.4039,  ..., 0.3098, 0.3059, 0.3020],
          [0.4392, 0.4118, 0.3608,  ..., 0.3294, 0.3216, 0.3176],
          [0.3529, 0.3333, 0.2863,  ...,

In [392]:
dataclass.images

tensor([[[[0.6039, 0.5686, 0.5059,  ..., 0.3137, 0.3098, 0.3059],
          [0.5529, 0.5176, 0.4588,  ..., 0.3333, 0.3255, 0.3216],
          [0.4549, 0.4353, 0.3843,  ..., 0.3373, 0.3333, 0.3294],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5608, 0.7294, 0.7647],
          [0.0314, 0.0392, 0.0549,  ..., 0.3686, 0.5255, 0.5843],
          [0.0196, 0.0314, 0.0510,  ..., 0.2275, 0.3608, 0.4431]],

         [[0.5137, 0.4902, 0.4235,  ..., 0.3059, 0.3020, 0.2980],
          [0.4627, 0.4392, 0.3843,  ..., 0.3255, 0.3176, 0.3137],
          [0.3725, 0.3529, 0.3098,  ..., 0.3294, 0.3255, 0.3216],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5020, 0.6549, 0.6902],
          [0.0314, 0.0392, 0.0549,  ..., 0.3098, 0.4510, 0.5098],
          [0.0196, 0.0314, 0.0510,  ..., 0.1686, 0.2863, 0.3686]],

         [[0.4902, 0.4627, 0.4039,  ..., 0.3098, 0.3059, 0.3020],
          [0.4392, 0.4118, 0.3608,  ..., 0.3294, 0.3216, 0.3176],
          [0.3529, 0.3333, 0.2863,  ..., 0

In [342]:
images

[tensor([[[0.6039, 0.5686, 0.5059,  ..., 0.3137, 0.3098, 0.3059],
          [0.5529, 0.5176, 0.4588,  ..., 0.3333, 0.3255, 0.3216],
          [0.4549, 0.4353, 0.3843,  ..., 0.3373, 0.3333, 0.3294],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5608, 0.7294, 0.7647],
          [0.0314, 0.0392, 0.0549,  ..., 0.3686, 0.5255, 0.5843],
          [0.0196, 0.0314, 0.0510,  ..., 0.2275, 0.3608, 0.4431]],
 
         [[0.5137, 0.4902, 0.4235,  ..., 0.3059, 0.3020, 0.2980],
          [0.4627, 0.4392, 0.3843,  ..., 0.3255, 0.3176, 0.3137],
          [0.3725, 0.3529, 0.3098,  ..., 0.3294, 0.3255, 0.3216],
          ...,
          [0.0471, 0.0510, 0.0549,  ..., 0.5020, 0.6549, 0.6902],
          [0.0314, 0.0392, 0.0549,  ..., 0.3098, 0.4510, 0.5098],
          [0.0196, 0.0314, 0.0510,  ..., 0.1686, 0.2863, 0.3686]],
 
         [[0.4902, 0.4627, 0.4039,  ..., 0.3098, 0.3059, 0.3020],
          [0.4392, 0.4118, 0.3608,  ..., 0.3294, 0.3216, 0.3176],
          [0.3529, 0.3333, 0.2863,  ...,

In [289]:
targets = []
counter = 0
for classifier_params in box:
    target = {}
    target['boxes'] = uh_oh_stinky[counter]
    target['labels'] = classifier_params
    targets.append(target)
    counter += 1

In [292]:
targets

[{'boxes': tensor([[186.7500, 318.5000, 213.2500, 341.5000],
          [171.7500, 296.0000, 199.2500, 318.0000],
          [164.7500, 272.0000, 193.2500, 290.0000],
          [234.0000, 178.5000, 263.0000, 196.5000]]),
  'labels': tensor([17., 24.,  8.,  8.])},
 {'boxes': tensor([[177.2500, 155.2500, 204.7500, 169.7500],
          [179.7500, 176.2500, 208.2500, 189.7500],
          [176.2500, 204.0000, 204.7500, 219.0000],
          [ 75.0000, 261.7500, 104.0000, 276.2500]]),
  'labels': tensor([36., 23., 44., 44.])},
 {'boxes': tensor([[262.7500,  58.7500, 287.2500,  80.2500],
          [271.7500,  77.5000, 298.2500,  98.5000],
          [281.0000, 115.5000, 308.0000, 130.5000],
          [205.2500, 204.7500, 233.7500, 220.2500]]),
  'labels': tensor([30., 31.,  6.,  6.])},
 {'boxes': tensor([[240.2500, 305.0000, 259.7500, 332.0000],
          [220.7500, 293.0000, 242.2500, 318.0000],
          [194.2500, 267.5000, 218.7500, 288.5000],
          [224.0000, 153.5000, 249.0000, 175.5000

In [346]:
datfer = DataLoader(shbi, batch_size=10, shuffle=True, collate_fn=collate_fn)

In [347]:
len(targets)

10

In [ ]:
images, uh_oh_stinky, bief)

In [357]:
images[0]

tensor([[[0.1608, 0.2196, 0.1216,  ..., 0.1804, 0.1804, 0.4118],
         [0.1765, 0.2235, 0.1882,  ..., 0.1725, 0.2118, 0.3490],
         [0.2118, 0.2353, 0.2510,  ..., 0.1529, 0.2510, 0.2980],
         ...,
         [0.0824, 0.0824, 0.0745,  ..., 0.3647, 0.4275, 0.4667],
         [0.0627, 0.0902, 0.1059,  ..., 0.4745, 0.5216, 0.5412],
         [0.0667, 0.0941, 0.1020,  ..., 0.5333, 0.5529, 0.5569]],

        [[0.1608, 0.2235, 0.1216,  ..., 0.1569, 0.1490, 0.3804],
         [0.1725, 0.2196, 0.1804,  ..., 0.1490, 0.1804, 0.3176],
         [0.2039, 0.2275, 0.2353,  ..., 0.1216, 0.2118, 0.2627],
         ...,
         [0.0745, 0.0745, 0.0667,  ..., 0.3608, 0.4235, 0.4627],
         [0.0549, 0.0824, 0.0980,  ..., 0.4706, 0.5176, 0.5373],
         [0.0588, 0.0863, 0.0941,  ..., 0.5294, 0.5490, 0.5529]],

        [[0.1922, 0.2431, 0.1216,  ..., 0.1098, 0.0980, 0.3294],
         [0.1961, 0.2392, 0.1843,  ..., 0.1020, 0.1294, 0.2667],
         [0.2157, 0.2314, 0.2314,  ..., 0.0784, 0.1647, 0.

In [350]:
uh_oh_stinky[0]

tensor([[186.7500, 318.5000, 213.2500, 341.5000],
        [171.7500, 296.0000, 199.2500, 318.0000],
        [164.7500, 272.0000, 193.2500, 290.0000],
        [234.0000, 178.5000, 263.0000, 196.5000]])

In [371]:
bhd = bief[0]
tensor = torch.tensor(bhd, dtype=torch.int64)

C:\Users\Obama\AppData\Local\Temp\ipykernel_30192\516235137.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(bhd, dtype=torch.int64)


In [372]:
targets = [{"boxes" : uh_oh_stinky[0], "labels" : tensor}]

In [365]:
hu = [images[0]]

In [373]:
output = model(hu, targets)

In [377]:
output

{'loss_classifier': tensor(4.2708, grad_fn=<NllLossBackward0>),
 'loss_box_reg': tensor(0.0072, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(1.0097, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(0.0598, grad_fn=<DivBackward0>)}

In [379]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [380]:
bhd2 = bief[1]
tensor2 = torch.tensor(bhd2, dtype=torch.int64)

targets2 = [{"boxes" : uh_oh_stinky[1], "labels" : tensor2}]

hu2 = [images[1]]

C:\Users\Obama\AppData\Local\Temp\ipykernel_30192\1652757256.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor2 = torch.tensor(bhd2, dtype=torch.int64)


In [385]:
output = model(hu2, targets2)

In [396]:
for i in range(100):
    bhd2 = bief[i]
    tensor = torch.tensor(bhd2, dtype=torch.int64)
    targets2 = [{"boxes" : uh_oh_stinky[i], "labels" : tensor}]
    hu2 = [images[i]]
    output = model(hu2, targets2)

C:\Users\Obama\AppData\Local\Temp\ipykernel_30192\2262343639.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(bhd2, dtype=torch.int64)


In [398]:
output

{'loss_classifier': tensor(4.2705, grad_fn=<NllLossBackward0>),
 'loss_box_reg': tensor(0.0288, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(0.9196, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(0.0271, grad_fn=<DivBackward0>)}

In [399]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [400]:
x = [images[200]]
predictions = model(x)

In [401]:
predictions

[{'boxes': tensor([[209.3927, 153.0201, 316.8363, 347.7866],
          [157.4268, 121.5111, 351.0114, 405.3015],
          [263.8635,   0.0000, 407.7886, 359.5119],
          [263.7250, 143.9498, 279.3160, 213.0556],
          [277.8052, 282.2362, 294.9158, 302.1543],
          [ 78.0967, 130.3044, 270.8938, 219.0146],
          [124.2942, 203.4650, 313.4895, 361.9333],
          [  0.0000,  43.2931, 254.8917, 171.9637],
          [268.8014, 141.9268, 286.6443, 218.7697],
          [126.7868,   0.0000, 405.3892, 202.5651],
          [223.9881,  10.6932, 346.7044, 240.2285],
          [120.6062,  10.5880, 416.0000, 212.3646],
          [  4.2005,  38.9892, 149.2936, 216.0623],
          [240.0831,  88.0563, 265.4396, 117.5403],
          [159.4387,  11.5570, 416.0000, 129.9746],
          [306.3655, 270.1770, 370.6447, 345.4498],
          [202.3208,  35.3295, 404.0099, 390.0704],
          [176.7187, 218.4967, 277.2127, 269.3734],
          [144.9183,  26.1988, 405.3919, 400.5684],
   